In [12]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

Copying gs://indaba-data/README.txt...
Copying gs://indaba-data/train/train.csv...
Copying gs://indaba-data/test/test.csv...
Copying gs://indaba-data/train/train_mut.pt...
Copying gs://indaba-data/train/train_wt.pt...
Copying gs://indaba-data/test/test_wt.pt...
Copying gs://indaba-data/test/test_mut.pt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").



In [13]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader


# move to data folder
%cd indaba-data

/content/indaba-data/indaba-data


In [14]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
df = pd.read_csv("train/train.csv")

In [15]:
len(df)

339778

In [16]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Split data into train and validation
wt_emb_train, wt_emb_val, mut_emb_train, mut_emb_val, df_train, df_val = train_test_split(wt_emb, mut_emb, df, test_size=0.21, random_state=42)

# Define the dataset class
class EmbeddingDataset(Dataset):
  def __init__(self, wt_pt, mut_pt, data_df):
    self.pt_wt = wt_pt
    self.pt_mut = mut_pt
    self.df = data_df

  def __len__(self):
    return len(self.pt_wt)

  def __getitem__(self, index):
    if "ddg" in self.df.columns:
      df_out = torch.Tensor([self.df.iloc[index]["ddg"]])
    else:
      df_out = torch.Tensor([self.df.iloc[index]["ID"]])

    return self.pt_wt[index,:], self.pt_mut[index,:], df_out

# Create separate datasets for the training and validation sets
train_dataset = EmbeddingDataset(wt_emb_train, mut_emb_train, df_train.reset_index(drop=True))
val_dataset = EmbeddingDataset(wt_emb_val, mut_emb_val, df_val.reset_index(drop=True))

# Create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [17]:
len(train_dataloader)

8389

In [18]:
len(val_dataloader)


2230

In [19]:
for i, (wt_emb, mut_emb, ddg) in enumerate(train_dataloader):
    print(f'wt_emb shape: {wt_emb.shape}, mut_emb shape: {mut_emb.shape}')
    if i > 5:  # Print shapes for first 5 batches only
        break


wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])
wt_emb shape: torch.Size([32, 1280]), mut_emb shape: torch.Size([32, 1280])


In [20]:
for wt_emb, mut_emb, ddg in train_dataloader:
    print(wt_emb_train.shape, mut_emb_train.shape, ddg.shape)
    break
import torch.nn as nn
import torch.nn.functional as F


torch.Size([268424, 1280]) torch.Size([268424, 1280]) torch.Size([32, 1])


In [21]:
class DeepFFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepFFNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        
        
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp2 = nn.Dropout(0.15)
        

        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, wt_emb, mut_emb):
        x = wt_emb - mut_emb  # subtracting embeddings
        
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dp2(x)
        
        
        out = self.fc4(x)
        
        return out

input_dim = 1280  # As we are subtracting wt_emb and mut_emb not concatenating
hidden_dim = 512
output_dim = 1  # ddG value

model = DeepFFNN(input_dim, hidden_dim, output_dim)


In [22]:
num_epochs = 20  # Adjust as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    train_losses = []

    for wt_emb, mut_emb, ddg in train_dataloader:
        wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
        optimizer.zero_grad()
        outputs = model(wt_emb, mut_emb)
        loss = criterion(outputs, ddg)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    with torch.no_grad():
        val_losses = []
        for wt_emb, mut_emb, ddg in val_dataloader:
            wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
            outputs = model(wt_emb, mut_emb)
            loss = criterion(outputs, ddg)
            val_losses.append(loss.item())
    
    avg_val_loss = sum(val_losses) / len(val_losses)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')


Epoch 1, Train Loss: 0.40813348780518593, Val Loss: 0.7508048354109307
Epoch 2, Train Loss: 0.39826357929687245, Val Loss: 0.7447785358525177
Epoch 3, Train Loss: 0.38785534792402415, Val Loss: 0.7549362494004681
Epoch 4, Train Loss: 0.3773500687165288, Val Loss: 0.7684580924265053
Epoch 5, Train Loss: 0.36884255184772996, Val Loss: 0.7645174865899065
Epoch 6, Train Loss: 0.36081609102817525, Val Loss: 0.7680147784707793
Epoch 7, Train Loss: 0.3521964849862215, Val Loss: 0.7737898040980502
Epoch 8, Train Loss: 0.3467534541868811, Val Loss: 0.7754447940352787
Epoch 9, Train Loss: 0.339283199818992, Val Loss: 0.7749598184374951
Epoch 10, Train Loss: 0.3312816337898457, Val Loss: 0.7718512967362532
Epoch 11, Train Loss: 0.3242235303205761, Val Loss: 0.7705528032686144
Epoch 12, Train Loss: 0.3201466804359848, Val Loss: 0.7810290714684088
Epoch 13, Train Loss: 0.31335820878693466, Val Loss: 0.7784994841958375
Epoch 14, Train Loss: 0.3082748122583457, Val Loss: 0.7822274666062385
Epoch 15, 

## Prediction & submission

In [23]:
# load embedding tensors & traing csv
wt_test_emb = torch.load("test/test_wt.pt")
mut_test_emb = torch.load("test/test_mut.pt")
df_test = pd.read_csv("test/test.csv")

In [24]:
# creating testing dataset and loading the embedding
test_dataset = EmbeddingDataset(wt_test_emb,mut_test_emb,df_test)
# preparing a dataloader for the testing
test_dataloader = torch.utils.data.dataloader.DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=2,
    )

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
df_result = pd.DataFrame()
with torch.no_grad():
  for batch_idx, (data_mut,data_wt , target) in tqdm(enumerate(test_dataloader)):
    x1 = data_wt.to(device)
    x2 = data_mut.to(device)
    id = target.to(device)
    # make prediction
    y_pred = model(x1,x2)
    df_result = pd.concat([df_result, pd.DataFrame({"ID":id.squeeze().cpu().numpy().astype(int) , "ddg" : y_pred.squeeze().cpu().numpy()})])

60it [00:01, 43.26it/s]


In [26]:
df_result.to_csv("submission_ahmed3.csv",index=False)